In [39]:
import yfinance
import numpy as np
import pandas as pd
import pandas
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import cufflinks as cf
cf.set_config_file(offline=True)

import warnings
warnings.filterwarnings("ignore")


In [40]:
from ActMng import AccountManager

In [41]:
starting_capital = 100000
allocation = 0.8

symbol = 'SPY'

long_trend_days = 12
short_trend_days = 6

In [42]:
def fetch_data(symbol:str='SPY', start_date:str='2020', end_date:str=None):
    
    if end_date is None:
        end = pandas.Timestamp.now()
    else:
        end = end_date
    
    return pdr.get_data_yahoo(symbol, pandas.Timestamp(start_date), pandas.Timestamp(end))[['Open','Close']]
df = fetch_data(symbol)

[*********************100%***********************]  1 of 1 completed


In [43]:
fast_trend = df.Open.rolling(window=short_trend_days).mean()
slow_trend = df.Open.rolling(window=long_trend_days).mean()

df['ma_fast'] = fast_trend
df['ma_slow'] = slow_trend
df[['Close','ma_fast','ma_slow']].iplot()

In [44]:
df['trade_direction'] = np.sign(fast_trend-slow_trend)
df

Open       Close     ma_fast     ma_slow  trade_direction
Date                                                                       
2019-12-31  320.529999  321.859985         NaN         NaN              NaN
2020-01-02  323.540009  324.869995         NaN         NaN              NaN
2020-01-03  321.160004  322.410004         NaN         NaN              NaN
2020-01-06  320.489990  323.640015         NaN         NaN              NaN
2020-01-07  323.019989  322.730011         NaN         NaN              NaN
...                ...         ...         ...         ...              ...
2021-11-05  469.279999  468.529999  462.054993  458.384997              1.0
2021-11-08  469.700012  468.929993  464.359996  459.879166              1.0
2021-11-09  469.320007  467.380005  465.863332  461.228333              1.0
2021-11-10  465.579987  463.619995  466.756663  462.169998              1.0
2021-11-11  465.209991  463.769989  467.408330  462.837496              1.0

[472 rows x 5 columns]

In [45]:
#Determine the direction of the moving average differences
df_trades = df['trade_direction'].dropna().diff().dropna().apply(np.sign).astype('int8')
df_trades = pandas.DataFrame(df_trades[df_trades !=0])

#Label our intended trades
df_trades['trade'] = df['trade_direction'].map({1:'Long',-1:'Short'})
df['trades'] = df_trades['trade']


In [46]:
#Label our opening trade after removing our window period
df['trades'].replace(pandas.np.NaN,'None',inplace=True)
df = df.dropna()
df['trades'].iloc[0]=('Long' if df['trade_direction'].iloc[0]==1.0 else 'Short')
df

Open       Close     ma_fast     ma_slow  trade_direction  \
Date                                                                          
2020-01-16  329.700012  330.920013  327.393341  324.670003              1.0   
2020-01-17  331.700012  331.950012  328.316676  325.600838              1.0   
2020-01-21  330.899994  331.299988  328.918340  326.214170              1.0   
2020-01-22  332.239990  331.339996  329.893336  327.137502              1.0   
2020-01-23  330.630005  331.720001  330.420003  327.982503              1.0   
...                ...         ...         ...         ...              ...   
2021-11-05  469.279999  468.529999  462.054993  458.384997              1.0   
2021-11-08  469.700012  468.929993  464.359996  459.879166              1.0   
2021-11-09  469.320007  467.380005  465.863332  461.228333              1.0   
2021-11-10  465.579987  463.619995  466.756663  462.169998              1.0   
2021-11-11  465.209991  463.769989  467.408330  462.837496              1.0   

           trades  
Date               
2020-01-16   Long  
2020-01-17   None  
2020-01-21   None  
2020-01-22   None  
2020-01-23   None  
...           ...  
2021-11-05   None  
2021-11-08   None  
2021-11-09   None  
2021-11-10   None  
2021-11-11   None  

[461 rows x 6 columns]

In [47]:
# Our trading strategy implementation - feel free to brose ActMng.py if you are curious.
trading_strategy = AccountManager(symbol=symbol,trades_df=df,starting_capital=starting_capital,allocation_amount=allocation)
results = trading_strategy.run()

In [48]:
#Here we can see a day by day breakdown of the account
results['result_df']

Open       Close     ma_fast     ma_slow  trade_direction  \
2020-01-16  329.700012  330.920013  327.393341  324.670003              1.0   
2020-01-17  331.700012  331.950012  328.316676  325.600838              1.0   
2020-01-21  330.899994  331.299988  328.918340  326.214170              1.0   
2020-01-22  332.239990  331.339996  329.893336  327.137502              1.0   
2020-01-23  330.630005  331.720001  330.420003  327.982503              1.0   
...                ...         ...         ...         ...              ...   
2021-11-05  469.279999  468.529999  462.054993  458.384997              1.0   
2021-11-08  469.700012  468.929993  464.359996  459.879166              1.0   
2021-11-09  469.320007  467.380005  465.863332  461.228333              1.0   
2021-11-10  465.579987  463.619995  466.756663  462.169998              1.0   
2021-11-11  465.209991  463.769989  467.408330  462.837496              1.0   

           trades    account_val       cash_bal  position  position_changes  \
2020-01-16   Long  100295.021096   20000.589038    242.64            242.64   
2020-01-17   None  100544.940000   20000.589038    242.64              0.00   
2020-01-21   None  100387.218076   20000.589038    242.64              0.00   
2020-01-22   None  100396.925750   20000.589038    242.64              0.00   
2020-01-23   None  100489.130134   20000.589038    242.64              0.00   
...           ...            ...            ...       ...               ...   
2021-11-05   None  136410.549112   25682.854500    236.33              0.00   
2021-11-08   None  136505.079669   25682.854500    236.33              0.00   
2021-11-09   None  136138.771054   25682.854500    236.33              0.00   
2021-11-10   None  135250.167946   25682.854500    236.33              0.00   
2021-11-11  Short  135285.616004  135284.616004      0.00           -236.33   

            fees  
2020-01-16   1.0  
2020-01-17   0.0  
2020-01-21   0.0  
2020-01-22   0.0  
2020-01-23   0.0  
...          ...  
2021-11-05   0.0  
2021-11-08   0.0  
2021-11-09   0.0  
2021-11-10   0.0  
2021-11-11   1.0  

[461 rows x 11 columns]

In [49]:
# Here are our simulated orders as we would submit to an API endpoint for algorthmic trading. 
results['broker_orders']

symbol                 date             time order order_type  \
2020-01-16    SPY  2020-01-16 00:00:00   At Market Open   BUY     MARKET   
2020-01-29    SPY  2020-01-29 00:00:00   At Market Open  SELL     MARKET   
2020-01-29    SPY  2020-01-29 00:00:00   At Market Open  SELL     MARKET   
2020-02-07    SPY  2020-02-07 00:00:00   At Market Open   BUY     MARKET   
2020-02-07    SPY  2020-02-07 00:00:00   At Market Open   BUY     MARKET   
...           ...                  ...              ...   ...        ...   
2021-09-14    SPY  2021-09-14 00:00:00   At Market Open  SELL     MARKET   
2021-09-14    SPY  2021-09-14 00:00:00   At Market Open  SELL     MARKET   
2021-10-13    SPY  2021-10-13 00:00:00   At Market Open   BUY     MARKET   
2021-10-13    SPY  2021-10-13 00:00:00   At Market Open   BUY     MARKET   
2021-11-11    SPY  2021-11-11 00:00:00  At Market Close  SELL     MARKET   

                 price    size  exchange_fees  
2020-01-16  329.700012  242.64            1.0  
2020-01-29  328.380005  242.64            1.0  
2020-01-29  328.380005  242.84            1.0  
2020-02-07  332.820007  242.84            1.0  
2020-02-07  332.820007  237.00            1.0  
...                ...     ...            ...  
2021-09-14  448.119995  223.86            1.0  
2021-09-14  448.119995  223.90            1.0  
2021-10-13  434.709991  223.90            1.0  
2021-10-13  434.709991  236.33            1.0  
2021-11-11  463.769989  236.33            1.0  

[66 rows x 8 columns]

In [50]:
#Todo:

result_df = results['result_df']

# Calculate the Absolute Return of our trend-following trading strategy
absolute_return_tf = (((result_df.account_val.iloc[-1])- starting_capital)/starting_capital)*100
print(f"absolute_return_tf: {np.round(absolute_return_tf,2)}% ")

# Calculate the CAGR or our trend-following trading strategy
CAGR_tf = ((result_df.account_val.iloc[-1]/starting_capital)**(252/len(df)))-1
print(f"CAGR_tf: {(np.round(CAGR_tf,2)*100)}%")

# Calculate the IR (Sharpe without RF-rate) of our trend-following trading strategy
information_ratio_tf = (result_df.account_val.pct_change().mean()/result_df.account_val.pct_change().std())*((252**0.5))
print(f"information_ratio_tf: {np.round(information_ratio_tf,4)}")

#Plot the equity curve (hint: use 'account_val')
result_df.account_val.plot()

absolute_return_tf: 35.29% 
CAGR_tf: 18.0%
information_ratio_tf: 0.9927


<AxesSubplot:>

In [51]:
#Todo:
# Calculate the Absolute Return of a buy-and-hold strategy in the same period
absolute_return_bh = ((result_df.Close.iloc[-1] - result_df.Open.iloc[0])/result_df.Open.iloc[0])*100
print(f"absolute_return_bh: {np.round(absolute_return_bh,2)}% ")

# Calculate the CAGR or our trend-following trading strategy
CAGR_bh = ((result_df.Close.iloc[-1]/result_df.Open.iloc[0])**(252/len(df)))-1
print(f"CAGR_bh: {(np.round(CAGR_bh,2)*100)}%")

# Calculate the IR (Sharpe without RF-rate) of our trend-following trading strategy
information_ratio_bh = (result_df.Close.pct_change().mean()/result_df.Close.pct_change().std())*((252**0.5))
print(f"information_ratio_bh: {np.round(information_ratio_bh,4)}")

#Plot the equity curve (hint: use 'close' returns and your initial capital value to generate an equity curve)
# Todo

absolute_return_bh: 40.66% 
CAGR_bh: 21.0%
information_ratio_bh: 0.837


In [ ]:
#Todo - optional, below:
# Copy any of the code from above to new cells below, and see how changing the symbol, dates and trend parameters changes the performance
# Can you beat the reference implementation?